In [1]:
!pip install kagglehub


In [2]:
import kagglehub
import os
import shutil
import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img

# Step 1 - Download the dataset
path = kagglehub.dataset_download("madhushreesannigrahi/fish-recognition-ground-truth-data")
print("Dataset base path:", path)

# Step 2 - Folder that contains fish_**
fish_image_path = os.path.join(path, "fish_image")

# Step 3 - Classes you want
selected_classes = ["fish_01", "fish_02", "fish_03", "fish_04", "fish_05", "fish_07"]

# Step 4 - Destination folder
new_dataset_path = "/content/fish_selected_6_classes.5"
os.makedirs(new_dataset_path, exist_ok=True)

# Create augmentation generator
aug_gen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True
)

TARGET_COUNT = 1000

# Step 5 - Process each class
for class_name in selected_classes:
    src = os.path.join(fish_image_path, class_name)
    dst = os.path.join(new_dataset_path, class_name)

    if not os.path.exists(src):
        print(f"Class not found: {class_name}")
        continue

    os.makedirs(dst, exist_ok=True)

    # Get all images in the source folder
    images = sorted(glob.glob(src + "/*"))

    # Copy first 2500 images (or all if less)
    copied_count = 0
    for img_path in images[:2500]:
        shutil.copy(img_path, dst)
        copied_count += 1

    print(f"Copied {copied_count} images from {class_name}")

    # If less than 1000 images → use augmentation
    if copied_count < TARGET_COUNT:
        needed = TARGET_COUNT - copied_count
        print(f"{class_name} needs augmentation: {needed} images")

        img_files = glob.glob(dst + "/*")  # existing (copied) images

        aug_count = 0
        idx = 0

        while aug_count < needed:
            img = load_img(img_files[idx])
            img_array = img_to_array(img)
            img_array = img_array.reshape((1,) + img_array.shape)

            # Generate 1 augmentation image at a time
            for batch in aug_gen.flow(img_array, batch_size=1,
                                      save_to_dir=dst,
                                      save_prefix="aug",
                                      save_format="jpg"):
                aug_count += 1
                break

            idx = (idx + 1) % len(img_files)

        print(f"Finished augmentation for {class_name}: now 1000 images total")

print("===== Dataset Processing Complete =====")


Using Colab cache for faster access to the 'fish-recognition-ground-truth-data' dataset.
Dataset base path: /kaggle/input/fish-recognition-ground-truth-data
Copied 2500 images from fish_01
Copied 2500 images from fish_02
Copied 2500 images from fish_03
Copied 2500 images from fish_04
Copied 2500 images from fish_05
Copied 450 images from fish_07
fish_07 needs augmentation: 550 images
Finished augmentation for fish_07: now 1000 images total
===== Dataset Processing Complete =====


In [3]:
from google.colab import drive
drive.mount('/content/drive')

save_dir = "/content/drive/MyDrive/fish_results.6c"      # where results will be saved
import os
os.makedirs(save_dir, exist_ok=True)


Mounted at /content/drive


In [4]:
import glob
import numpy as np
import os

img_paths = []
labels = []

# Make sure paths are correct
classes = sorted(os.listdir(new_dataset_path))

# All image extensions you want to include
extensions = ["*.jpg", "*.jpeg", "*.png", "*.JPG", "*.JPEG", "*.PNG"]

for class_name in classes:
    class_folder = os.path.join(new_dataset_path, class_name)

    for ext in extensions:
        for img_file in glob.glob(os.path.join(class_folder, ext)):
            img_paths.append(img_file)
            labels.append(class_name)

img_paths = np.array(img_paths)
labels = np.array(labels)

print("Total images:", len(img_paths))
print("Classes:", classes)



Total images: 13483
Classes: ['fish_01', 'fish_02', 'fish_03', 'fish_04', 'fish_05', 'fish_07']


In [5]:
from sklearn.preprocessing import LabelEncoder

label_enc = LabelEncoder()
labels_encoded = label_enc.fit_transform(labels)

num_classes = len(label_enc.classes_)
print("Number of classes:", num_classes)


Number of classes: 6


In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models

img_size = (128, 128)

def build_cnn_without_dropout():
    model = models.Sequential([
        layers.Input(shape=img_size + (3,)),

        layers.Conv2D(64, 3, activation="relu"),
        layers.MaxPooling2D(),

        layers.Conv2D(128, 3, activation="relu"),
        layers.MaxPooling2D(),

        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dense(num_classes, activation="softmax")
    ])
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model


def build_cnn_with_dropout():
    model = models.Sequential([
        layers.Input(shape=img_size + (3,)),

        layers.Conv2D(64, 3, activation="relu"),
        layers.MaxPooling2D(),

        layers.Conv2D(128, 3, activation="relu"),
        layers.MaxPooling2D(),

        layers.Flatten(),

        layers.Dense(128, activation="relu"),
        layers.Dropout(0.5),

        layers.Dense(num_classes, activation="softmax")
    ])
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model


In [7]:
import cv2

def load_image(path):
    img = cv2.imread(path)
    img = cv2.resize(img, img_size)
    return img


In [8]:
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import matplotlib.pyplot as plt

def run_cross_validation(model_builder, model_name):

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    fold = 1

    for train_idx, test_idx in kf.split(img_paths):

        print(f"\n==============================")
        print(f"    {model_name} - Fold {fold}")
        print(f"==============================")

        # Load training and test images
        X_train = np.array([load_image(p) for p in img_paths[train_idx]])
        X_test  = np.array([load_image(p) for p in img_paths[test_idx]])

        y_train = labels_encoded[train_idx]
        y_test  = labels_encoded[test_idx]

        model = model_builder()

        history = model.fit(
            X_train, y_train,
            epochs=10,
            validation_split=0.2,
            verbose=1
        )

        # Predictions
        y_pred = np.argmax(model.predict(X_test), axis=1)

        # Confusion matrix
        cm = confusion_matrix(y_test, y_pred)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_enc.classes_)

        plt.figure(figsize=(6,5))
        disp.plot(cmap="Blues", xticks_rotation=45)
        plt.title(f"{model_name} - Fold {fold} Confusion Matrix")
        plt.savefig(f"{save_dir}/{model_name}_fold{fold}_cm.png")
        plt.close()

        # Accuracy plots
        plt.figure()
        plt.plot(history.history["accuracy"], label="Train Acc")
        plt.plot(history.history["val_accuracy"], label="Val Acc")
        plt.legend()
        plt.title(f"{model_name} - Fold {fold} Accuracy")
        plt.savefig(f"{save_dir}/{model_name}_fold{fold}_acc.png")
        plt.close()

        # LOSS CURVE
        plt.figure()
        plt.plot(history.history["loss"], label="Train Loss")
        plt.plot(history.history["val_loss"], label="Val Loss")
        plt.title(f"{model_name} - Fold {fold} Loss")
        plt.xlabel("Epochs")
        plt.ylabel("Loss")
        plt.legend()
        plt.savefig(f"{save_dir}/{model_name}_Fold{fold}_Loss.png")
        plt.close()

        # Classification report
        report = classification_report(y_test, y_pred, target_names=label_enc.classes_)
        with open(f"{save_dir}/{model_name}_fold{fold}_report.txt", "w") as f:
            f.write(report)

        # Save model weights
        model.save(f"{save_dir}/{model_name}_fold{fold}.h5")

        fold += 1


In [19]:
run_cross_validation(build_cnn_without_dropout, "CNN_No_Dropout")


    CNN_No_Dropout - Fold 1
Epoch 1/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 368s 1s/step - accuracy: 0.6675 - loss: 114.3944 - val_accuracy: 0.5183 - val_loss: 8.3764
Epoch 2/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 380s 1s/step - accuracy: 0.9032 - loss: 0.2955 - val_accuracy: 0.5600 - val_loss: 12.7409
Epoch 3/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 390s 1s/step - accuracy: 0.9374 - loss: 0.1915 - val_accuracy: 0.4961 - val_loss: 5.0210
Epoch 4/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 347s 1s/step - accuracy: 0.9159 - loss: 0.2674 - val_accuracy: 0.5981 - val_loss: 38.6992
Epoch 5/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 366s 1s/step - accuracy: 0.9442 - loss: 0.1903 - val_accuracy: 0.6194 - val_loss: 11.5201
Epoch 6/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 373s 1s/step - accuracy: 0.9689 - loss: 0.1095 - val_accuracy: 0.5665 - val_loss: 35.2153
Epoch 7/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 357s 1s/step - accuracy: 0.9843 - loss: 0.0510 - val_accuracy: 0.6171 - val_loss: 22.4206
Epoch 8/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 367s 1s/step - accura

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



    CNN_No_Dropout - Fold 2
Epoch 1/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 382s 1s/step - accuracy: 0.6519 - loss: 53.8272 - val_accuracy: 0.5688 - val_loss: 2.5765
Epoch 2/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 370s 1s/step - accuracy: 0.8793 - loss: 0.3677 - val_accuracy: 0.6036 - val_loss: 2.9282
Epoch 3/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 367s 1s/step - accuracy: 0.9368 - loss: 0.1890 - val_accuracy: 0.6110 - val_loss: 4.2709
Epoch 4/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 367s 1s/step - accuracy: 0.9649 - loss: 0.1156 - val_accuracy: 0.5846 - val_loss: 8.1242
Epoch 5/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 365s 1s/step - accuracy: 0.9803 - loss: 0.0655 - val_accuracy: 0.5475 - val_loss: 4.6351
Epoch 6/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 372s 1s/step - accuracy: 0.9647 - loss: 0.1381 - val_accuracy: 0.6115 - val_loss: 9.8567
Epoch 7/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 367s 1s/step - accuracy: 0.9817 - loss: 0.0571 - val_accuracy: 0.5939 - val_loss: 5.2246
Epoch 8/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 348s 1s/step - accuracy: 0.

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



    CNN_No_Dropout - Fold 3
Epoch 1/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 363s 1s/step - accuracy: 0.6541 - loss: 124.6292 - val_accuracy: 0.5739 - val_loss: 3.8495
Epoch 2/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 388s 1s/step - accuracy: 0.8524 - loss: 0.4668 - val_accuracy: 0.5503 - val_loss: 2.1165
Epoch 3/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 364s 1s/step - accuracy: 0.8994 - loss: 0.3026 - val_accuracy: 0.5378 - val_loss: 3.7322
Epoch 4/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 381s 1s/step - accuracy: 0.9568 - loss: 0.1254 - val_accuracy: 0.5447 - val_loss: 8.8494
Epoch 5/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 355s 1s/step - accuracy: 0.9791 - loss: 0.0751 - val_accuracy: 0.5846 - val_loss: 8.7743
Epoch 6/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 372s 1s/step - accuracy: 0.9427 - loss: 0.2302 - val_accuracy: 0.5990 - val_loss: 8.6381
Epoch 7/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 353s 1s/step - accuracy: 0.9669 - loss: 0.1139 - val_accuracy: 0.5971 - val_loss: 8.6467
Epoch 8/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 351s 1s/step - accuracy: 0

...

<Figure size 600x500 with 0 Axes>

<Figure size 600x500 with 0 Axes>

<Figure size 600x500 with 0 Axes>

<Figure size 600x500 with 0 Axes>

<Figure size 600x500 with 0 Axes>

In [9]:
run_cross_validation(build_cnn_with_dropout, "CNN_With_Dropout")


    CNN_With_Dropout - Fold 1
Epoch 1/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 19s 48ms/step - accuracy: 0.6693 - loss: 60.4034 - val_accuracy: 0.6103 - val_loss: 2.2042
Epoch 2/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step - accuracy: 0.8821 - loss: 0.3522 - val_accuracy: 0.5556 - val_loss: 4.1490
Epoch 3/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step - accuracy: 0.9258 - loss: 0.2308 - val_accuracy: 0.6075 - val_loss: 4.2063
Epoch 4/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 8s 30ms/step - accuracy: 0.9201 - loss: 0.2500 - val_accuracy: 0.5802 - val_loss: 5.4423
Epoch 5/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step - accuracy: 0.9307 - loss: 0.2466 - val_accuracy: 0.4657 - val_loss: 5.4123
Epoch 6/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step - accuracy: 0.8351 - loss: 0.5285 - val_accuracy: 0.4829 - val_loss: 8.7761
Epoch 7/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 8s 30ms/step - accuracy: 0.8960 - loss: 0.2990 - val_accuracy: 0.6047 - val_loss: 13.9417
Epoch 8/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step - accuracy

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



    CNN_With_Dropout - Fold 2
Epoch 1/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 14s 41ms/step - accuracy: 0.5866 - loss: 55.9053 - val_accuracy: 0.5385 - val_loss: 2.1179
Epoch 2/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step - accuracy: 0.8045 - loss: 0.6297 - val_accuracy: 0.5366 - val_loss: 2.6281
Epoch 3/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step - accuracy: 0.8765 - loss: 0.3785 - val_accuracy: 0.5218 - val_loss: 3.5060
Epoch 4/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step - accuracy: 0.9036 - loss: 0.3453 - val_accuracy: 0.5723 - val_loss: 9.0356
Epoch 5/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step - accuracy: 0.9118 - loss: 0.2681 - val_accuracy: 0.5653 - val_loss: 4.6879
Epoch 6/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 8s 30ms/step - accuracy: 0.9097 - loss: 0.2900 - val_accuracy: 0.5894 - val_loss: 8.9674
Epoch 7/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step - accuracy: 0.9518 - loss: 0.1611 - val_accuracy: 0.5570 - val_loss: 6.9883
Epoch 8/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step - accuracy:

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



    CNN_With_Dropout - Fold 3
Epoch 1/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 14s 41ms/step - accuracy: 0.5516 - loss: 83.4289 - val_accuracy: 0.3758 - val_loss: 3.5889
Epoch 2/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 9s 31ms/step - accuracy: 0.8236 - loss: 0.5290 - val_accuracy: 0.5454 - val_loss: 2.7522
Epoch 3/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step - accuracy: 0.8738 - loss: 0.3817 - val_accuracy: 0.5982 - val_loss: 5.2283
Epoch 4/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step - accuracy: 0.8997 - loss: 0.3169 - val_accuracy: 0.5310 - val_loss: 4.0408
Epoch 5/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step - accuracy: 0.9101 - loss: 0.2631 - val_accuracy: 0.5797 - val_loss: 13.3946
Epoch 6/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step - accuracy: 0.9394 - loss: 0.1862 - val_accuracy: 0.6057 - val_loss: 8.5509
Epoch 7/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step - accuracy: 0.9558 - loss: 0.1375 - val_accuracy: 0.6033 - val_loss: 12.4756
Epoch 8/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 8s 30ms/step - accurac

...

<Figure size 600x500 with 0 Axes>

<Figure size 600x500 with 0 Axes>

<Figure size 600x500 with 0 Axes>

<Figure size 600x500 with 0 Axes>

<Figure size 600x500 with 0 Axes>